In [163]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=iphone+12&i=electronics&rh=n%3A560820%2Cp_72%3A419153031&dc&page=2&qid=1698910578&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Apple-iPhone-12-256GB-Blue/dp/B08PCFF5QB/ref=sr_1_27?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-27',
 '/Apple-iPhone-mini-128GB-Green/dp/B08PCFKLHM/ref=sr_1_40?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-40',
 '/Apple-iPhone-128GB-Pacific-Blue/dp/B08PCCKHJM/ref=sr_1_44?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-44',
 '/Apple-iPhone-256GB-Pacific-Blue/dp/B08PCC2Y59/ref=sr_1_39?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-39',
 '/Apple-iPhone-256GB-Pacific-Blue/dp/B08PCD1YDK/ref=sr_1_41?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-41',
 '/Apple-iPhone-128GB-Starlight-Renewed/dp/B09MGDRQSS/ref=sr_1_47?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-47',
 '/Apple-iPhone-SE-64GB-White/dp/B08

In [164]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPhone'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-iPhone-12-256GB-Blue/dp/B08PCFF5QB/ref=sr_1_27?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-27
2 / 240
/Apple-iPhone-mini-128GB-Green/dp/B08PCFKLHM/ref=sr_1_40?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-40
3 / 240
/Apple-iPhone-128GB-Pacific-Blue/dp/B08PCCKHJM/ref=sr_1_44?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-44
4 / 240
/Apple-iPhone-256GB-Pacific-Blue/dp/B08PCC2Y59/ref=sr_1_39?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-39
5 / 240
/Apple-iPhone-256GB-Pacific-Blue/dp/B08PCD1YDK/ref=sr_1_41?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-41
6 / 240
/Apple-iPhone-128GB-Starlight-Renewed/dp/B09MGDRQSS/ref=sr_1_47?keywords=iphone+12&qid=1698910599&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-47
7 / 240
/App

51 / 240
/Apple-iPhone-64GB-White-Renewed/dp/B082BJLRB5/ref=sr_1_35?keywords=iphone+12&qid=1698910610&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-35
52 / 240
/Apple-iPhone-mini-64GB-White/dp/B08PCDBHFF/ref=sr_1_26?keywords=iphone+12&qid=1698910610&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-26
53 / 240
/New-Apple-iPhone-12-128GB/dp/B08L5RD3KR/ref=sr_1_36?keywords=iphone+12&qid=1698910610&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-36
54 / 240
/New-Apple-iPhone-12-128GB/dp/B08L5P5B68/ref=sr_1_31?keywords=iphone+12&qid=1698910610&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-31
55 / 240
/Apple-iPhone-128GB-Starlight-Renewed/dp/B09MGDRQSS/ref=sr_1_47?keywords=iphone+12&qid=1698910610&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-47
56 / 240
/Apple-iPhone-mini-256GB-White/dp/B08PCFC3C1/ref=sr_1_30?keywords=iphone+12&qid=1698910610&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-30
57 / 240
/Apple-

101 / 240
/Apple-iPhone-256GB-Pacific-Blue/dp/B08PCC2Y59/ref=sr_1_39?keywords=iphone+12&qid=1698910620&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-39
102 / 240
/Apple-iPhone-mini-128GB-Green/dp/B08PCFKLHM/ref=sr_1_43?keywords=iphone+12&qid=1698910620&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-43
103 / 240
/Apple-iPhone-14-128-GB/dp/B0BDJC83QY/ref=sr_1_46?keywords=iphone+12&qid=1698910620&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-46
104 / 240
/Apple-iPhone-mini-256GB-White/dp/B08PCFC3C1/ref=sr_1_30?keywords=iphone+12&qid=1698910620&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-30
105 / 240
/Apple-iPhone-128GB-Starlight-Renewed/dp/B09MGDRQSS/ref=sr_1_47?keywords=iphone+12&qid=1698910620&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-47
106 / 240
/Apple-iPhone-13-128GB-Starlight/dp/B09G9FB7LV/ref=sr_1_25?keywords=iphone+12&qid=1698910620&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-25
107 / 

151 / 240
/Apple-iPhone-64GB-Yellow-Renewed/dp/B082BHMBG7/ref=sr_1_37?keywords=iphone+12&qid=1698910631&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-37
152 / 240
/Apple-iPhone-256GB-Product-Red/dp/B08PCFMXN4/ref=sr_1_48?keywords=iphone+12&qid=1698910631&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-48
153 / 240
/Apple-iPhone-mini-64GB-White/dp/B08PCDBHFF/ref=sr_1_26?keywords=iphone+12&qid=1698910631&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-26
154 / 240
/Apple-iPhone-13-128GB-Blue/dp/B09G9HQHRH/ref=sr_1_43?keywords=iphone+12&qid=1698910631&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-43
155 / 240
/New-Apple-iPhone-12-128GB/dp/B08L5RD3KR/ref=sr_1_36?keywords=iphone+12&qid=1698910631&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-36
156 / 240
/Apple-iPhone-128GB-Pacific-Blue/dp/B08PCCKHJM/ref=sr_1_44?keywords=iphone+12&qid=1698910631&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-44
157 / 240
/A

201 / 240
/Apple-iPhone-14-128-GB/dp/B0BDJC83QY/ref=sr_1_46?keywords=iphone+12&qid=1698910641&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-46
202 / 240
/Apple-iPhone-12-256GB-Blue/dp/B08PCFF5QB/ref=sr_1_27?keywords=iphone+12&qid=1698910641&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-27
203 / 240
/Apple-iPhone-SE-64GB-White/dp/B08L6XJX3M/ref=sr_1_32?keywords=iphone+12&qid=1698910641&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-32
204 / 240
/Apple-iPhone-64GB-White-Renewed/dp/B082BJLRB5/ref=sr_1_35?keywords=iphone+12&qid=1698910641&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-35
205 / 240
/Apple-iPhone-256GB-Product-Red/dp/B08PCFMXN4/ref=sr_1_48?keywords=iphone+12&qid=1698910641&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-48
206 / 240
/Apple-iPhone-128GB-Red-Renewed/dp/B09MGFMDFN/ref=sr_1_40?keywords=iphone+12&qid=1698910641&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-40
207 / 240
/Apple-iP

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Mounting type,Unit count,Controller type,Cable type,Compatible phone models,Material,Pattern,Embellishment,Theme,Included components
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,"Apple iPhone 12 Pro Max, 128GB, Pacific Blue (...",414..,4.1 out of 5,"2,041 global ratings",https://www.amazon.co.uk/Apple-iPhone-128GB-Pa...,Value not found,Apple,iPhone 12 Pro Max,iOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,"Apple iPhone 12 Pro Max, 128GB, Pacific Blue (...",414..,4.1 out of 5,"2,041 global ratings",https://www.amazon.co.uk/Apple-iPhone-128GB-Pa...,Value not found,Apple,iPhone 12 Pro Max,iOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,"Apple iPhone 12 Pro Max, 128GB, Pacific Blue (...",414..,4.1 out of 5,"2,041 global ratings",https://www.amazon.co.uk/Apple-iPhone-128GB-Pa...,Value not found,Apple,iPhone 12 Pro Max,iOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,"Apple iPhone 12 Pro Max, 128GB, Pacific Blue (...",414..,4.1 out of 5,"2,041 global ratings",https://www.amazon.co.uk/Apple-iPhone-128GB-Pa...,Value not found,Apple,iPhone 12 Pro Max,iOS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
df

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Mounting type,Unit count,Controller type,Cable type,Compatible phone models,Material,Pattern,Embellishment,Theme,Included components
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,Apple iPhone 14 Pro Max (128 GB) - Space Black,Price not found,4.6 out of 5,"1,226 global ratings",https://www.amazon.co.uk/Apple-iPhone-Pro-Max-...,Value not found,Apple,IPhone 14 Pro Max,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,Apple iPhone 14 Pro Max (128 GB) - Space Black,Price not found,4.6 out of 5,"1,226 global ratings",https://www.amazon.co.uk/Apple-iPhone-Pro-Max-...,Value not found,Apple,IPhone 14 Pro Max,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,Apple iPhone 14 Pro Max (128 GB) - Space Black,Price not found,4.6 out of 5,"1,226 global ratings",https://www.amazon.co.uk/Apple-iPhone-Pro-Max-...,Value not found,Apple,IPhone 14 Pro Max,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,Apple iPhone 14 Pro Max (128 GB) - Space Black,Price not found,4.6 out of 5,"1,226 global ratings",https://www.amazon.co.uk/Apple-iPhone-Pro-Max-...,Value not found,Apple,IPhone 14 Pro Max,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
